In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, split, explode, count, lower

In [2]:
spark = SparkSession \
    .builder \
    .appName("Databases II") \
    .getOrCreate()

In [3]:
movie = (spark.read
      .format("csv")
      .option('header', 'true')
      .option("delimiter", ",")
      .option("inferSchema", "true")
      .load("movie.csv")
     )

In [4]:
rating = (spark.read
      .format("csv")
      .option('header', 'true')
      .option("delimiter", ",")
      .option("inferSchema", "true")
      .load("rating.csv")
     )

In [5]:
tag = (spark.read
      .format("csv")
      .option('header', 'true')
      .option("delimiter", ",")
      .option("inferSchema", "true")
      .load("tag.csv")
     )

In [6]:
# change the strings of column 'tag' to lowercase
tag = tag.select((col('userId')),
                  col('movieId'),
                  lower(col('tag')).alias('lower_tag'),
                  col('timestamp'))

In [7]:
# execute inner join between 'rating' and 'tag' dataframes
joined = rating.join(tag, (rating.userId == tag.userId) & (rating.movieId == tag.movieId), 'inner')

In [8]:
# from dataframe 'joined' select only the rows whose 'lower_tag' column contains the word funny
# and whose 'rating' column is > 3.5
funny_movies = joined.filter((joined['lower_tag'].contains('funny')) & (~joined["lower_tag"].contains('no')) & (joined["rating"] > 3.5))

In [9]:
# because some movies have more than one genres listed,
# we split the column 'genres' with the '|' operator so we can process on each genre separately
# and then use explode so that every distinct genre will be in their own row
movie = movie.withColumn("genres", explode(split("genres", "[|]")))

In [10]:
# execute inner join between 'movie' and 'funny_movies' dataframes
joined = movie.join(funny_movies, ["movieId"], 'inner')

In [11]:
# create new dataframe 'genre' with the columns of dataframe 'joined'
# grouped by the column 'genres'
# and count the number of distinct movies for each genre
genre = joined.select('*').groupby(joined["genres"]).agg(count('title').alias('movies_count'))

In [12]:
# show the results ordered by alphabetical order of column 'genres'
query10 = genre.orderBy(genre["genres"].asc()).show()

+-----------+------------+
|     genres|movies_count|
+-----------+------------+
|     Action|         431|
|  Adventure|         465|
|  Animation|         268|
|   Children|         273|
|     Comedy|        1618|
|      Crime|         276|
|Documentary|          27|
|      Drama|         544|
|    Fantasy|         306|
|  Film-Noir|           3|
|     Horror|         140|
|       IMAX|          74|
|    Musical|          92|
|    Mystery|          93|
|    Romance|         490|
|     Sci-Fi|         197|
|   Thriller|         236|
|        War|          37|
|    Western|          44|
+-----------+------------+



In [13]:
spark.stop()